In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey, Table
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import sys
sys.path.append('../')
from modules import SqlApp

In [2]:
s = SqlApp()

In [3]:
s.connect('postgresql://postgres@localhost:5532/sqlapp')

In [4]:
populate = s.populate
create = s.create
Base = s.base

In [7]:
# /!\ DROP MODEL IN DB /!\
s.drop_models()

## Many To One Relationship

In [8]:
"""
A basic Many to One Relationship
"""

class PlayerModel(Base):
    """Data model for players."""
    __tablename__ = "players"

    id = Column(Integer,
                primary_key=True,
                nullable=False)
    team_id = Column(Integer,
                     ForeignKey('teams.id'),
                     nullable=False)
    name = Column(String(100),
                  nullable=False)
    position = Column(String(100),
                      nullable=False)

    # Relationships
    team = relationship("TeamModel", backref="players")
    
    def __repr__(self):
        return '<Person model {}>'.format(self.id)


class TeamModel(Base):
    """Data model for people."""
    __tablename__ = "teams"

    id = Column(Integer,
                primary_key=True,
                nullable=False,
    )
    name = Column(String(100),
                  nullable=False)
    city = Column(String(100),
                  nullable=False)

    def __repr__(self):
        return '<Team model {}>'.format(self.id)

In [9]:
# PlayerModel.__table__.drop(engine)
# TeamModel.__table__.drop(engine)

# Many 2 Many Relationship

In [10]:


association_table = Table('association', Base.metadata,
    Column('left_id', Integer, ForeignKey('left.id')),
    Column('right_id', String, ForeignKey('right.id'))
)


class Parent(Base):
    __tablename__ = 'left'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    children = relationship(
        "Child",
        secondary=association_table,
        back_populates="parents")


class Child(Base):
    __tablename__ = 'right'
    id = Column(String, primary_key=True, )
    parents = relationship(
        "Parent",
        secondary=association_table,
        back_populates="children")

In [12]:
# /!\ WRITE MODEL IN DB /!\
# Base.metadata.create_all(engine)
s.create_models()

2020-02-04 19:04:56,321 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-02-04 19:04:56,323 INFO sqlalchemy.engine.base.Engine {'name': u'players'}
2020-02-04 19:04:56,335 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-02-04 19:04:56,337 INFO sqlalchemy.engine.base.Engine {'name': u'left'}
2020-02-04 19:04:56,341 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-02-04 19:04:56,343 INFO sqlalchemy.engine.base.Engine {'name': u'right'}
2020-02-04 19:04:56,347 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visib

In [13]:
TEAMS = [
    { 'name': 'PSG', 'city': 'Paris', 'id': 1},
    { 'name': 'OM', 'city': 'Marseille', 'id': 2},
    { 'name': 'OL', 'city': 'Lyon', 'id': 3},
    { 'name': 'LOSC', 'city': 'Lille', 'id': 4},
]
populate(TeamModel, TEAMS)

2020-02-04 19:05:07,009 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 19:05:07,012 INFO sqlalchemy.engine.base.Engine INSERT INTO teams (id, name, city) VALUES (%(id)s, %(name)s, %(city)s)
2020-02-04 19:05:07,014 INFO sqlalchemy.engine.base.Engine ({'city': 'Paris', 'id': 1, 'name': 'PSG'}, {'city': 'Marseille', 'id': 2, 'name': 'OM'}, {'city': 'Lyon', 'id': 3, 'name': 'OL'}, {'city': 'Lille', 'id': 4, 'name': 'LOSC'})
2020-02-04 19:05:07,027 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
PLAYERS = [
    {'name':'Mbappe', 'position': '9', 'team_id': '1'},
    {'name':'Neymar', 'position': '7', 'team_id': '1'},
    {'name':'Diawara', 'position': '8', 'team_id': '3'},
    {'name':'Benedeto', 'position': '9', 'team_id': '2'},
    {'name':'Payet', 'position': '7', 'team_id': '2'},
    {'name':'Pepe', 'position': '10', 'team_id': '4'},
]
populate(PlayerModel, PLAYERS)

2020-02-04 19:05:11,331 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 19:05:11,334 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-04 19:05:11,336 INFO sqlalchemy.engine.base.Engine {'position': '9', 'team_id': '1', 'name': 'Mbappe'}
2020-02-04 19:05:11,344 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-04 19:05:11,345 INFO sqlalchemy.engine.base.Engine {'position': '7', 'team_id': '1', 'name': 'Neymar'}
2020-02-04 19:05:11,349 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-04 19:05:11,350 INFO sqlalchemy.engine.base.Engine {'position': '8', 'team_id': '3', 'name': 'Diawara'}
2020-02-04 19:05:11,357 INFO sqlalchemy.engine.base.Engine INSERT INTO player

In [15]:
populate(Parent, [{'id':1, 'name': 'x'}, {'id':2, 'name': 'y'}])
populate(Child, [
    {'id': 'FR',}, {'id': 'ES'}, {'id': 'IT'},{'id': 'EN'}
])

2020-02-04 19:05:15,193 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 19:05:15,195 INFO sqlalchemy.engine.base.Engine INSERT INTO "left" (id, name) VALUES (%(id)s, %(name)s)
2020-02-04 19:05:15,198 INFO sqlalchemy.engine.base.Engine ({'id': 1, 'name': 'x'}, {'id': 2, 'name': 'y'})
2020-02-04 19:05:15,216 INFO sqlalchemy.engine.base.Engine COMMIT
2020-02-04 19:05:15,225 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 19:05:15,227 INFO sqlalchemy.engine.base.Engine INSERT INTO "right" (id) VALUES (%(id)s)
2020-02-04 19:05:15,229 INFO sqlalchemy.engine.base.Engine ({'id': 'FR'}, {'id': 'ES'}, {'id': 'IT'}, {'id': 'EN'})
2020-02-04 19:05:15,245 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
p = s.session.query(Parent).filter_by(id=1).first()

2020-02-04 19:05:18,399 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-04 19:05:18,401 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id, "left".name AS left_name 
FROM "left" 
WHERE "left".id = %(id_1)s 
 LIMIT %(param_1)s
2020-02-04 19:05:18,403 INFO sqlalchemy.engine.base.Engine {'id_1': 1, 'param_1': 1}


In [17]:
p.children

2020-02-04 19:05:19,214 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right", association 
WHERE %(param_1)s = association.left_id AND "right".id = association.right_id
2020-02-04 19:05:19,216 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[]

In [18]:
p.children.append(s.session.query(Child).filter_by(id='ES').first())

2020-02-04 19:05:21,952 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right" 
WHERE "right".id = %(id_1)s 
 LIMIT %(param_1)s
2020-02-04 19:05:21,954 INFO sqlalchemy.engine.base.Engine {'id_1': 'ES', 'param_1': 1}


In [19]:
create(p)

2020-02-04 19:05:22,540 INFO sqlalchemy.engine.base.Engine INSERT INTO association (left_id, right_id) VALUES (%(left_id)s, %(right_id)s)
2020-02-04 19:05:22,543 INFO sqlalchemy.engine.base.Engine {'left_id': 1, 'right_id': u'ES'}
2020-02-04 19:05:22,549 INFO sqlalchemy.engine.base.Engine COMMIT
